In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from utilities import *
from keras import backend as K
from tensorflow.keras import layers, regularizers
import keras
import utils.augmentation as aug
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, dataset = load_larger_dataset_first_scenario()
train_set_y = transform_label(train_set_y_orig)
test_set_y = transform_label(test_set_y_orig)
Y = pd.get_dummies(train_set_y)
Y = Y.replace({True: 1, False: 0}).astype(float)

from sklearn.preprocessing import StandardScaler

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to your dataframe and transform it
train_x_std = scaler.fit_transform(train_set_x_orig)
train_x = pd.DataFrame(train_x_std)

scaler = StandardScaler()
# fit the scaler to your dataframe and transform it
test_x_std = scaler.fit_transform(test_set_x_orig)
test_x = pd.DataFrame(test_x_std)
test_set_y = transform_label(test_set_y_orig)

2023-04-30 14:47:22.454418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Add augmented data into training set
augmentation_train_set = augment_training_set(train_x, train_set_y)
augmentation_train_x = augmentation_train_set.iloc[:, :1000]
augmentation_train_y = augmentation_train_set.iloc[:, 1000]

augmentation_train_x


,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.535068,0.224282,-0.148606,-0.343657,-0.641576,-0.832988,-0.849029,-0.744321,-0.828458,-0.548708,...,-0.886991,-1.583172,-1.930889,-1.827324,-1.647419,-1.374463,-0.844228,-0.249675,0.110226,0.724039
1,-1.499316,-1.648444,-1.646612,-1.490652,-1.129920,-0.743790,-0.354456,0.030432,0.396167,0.717596,...,-1.206499,-1.225544,-1.184891,-1.144239,-1.103587,-1.057802,-0.954709,-0.851617,-0.748524,-0.645431
2,0.608131,-0.050384,-0.616007,-0.035738,0.716734,0.455001,0.054995,0.368301,0.901938,0.914435,...,0.255456,0.406873,0.501187,0.464024,0.291978,0.051802,-0.120632,-0.246743,-0.211486,-0.243234
3,0.412884,0.699084,0.827273,0.939424,0.866684,0.674207,0.750674,1.066309,1.165676,1.299208,...,0.835677,1.359029,1.711895,2.120469,2.528132,2.576589,2.508295,2.659422,2.750255,2.705510
4,-0.199303,-0.003597,0.170967,0.362338,0.450999,0.507196,0.510352,0.451951,0.388107,0.409840,...,-0.591117,-0.975192,-1.172449,-1.211592,-1.144271,-1.008544,-0.813823,-0.585806,-0.318387,-0.073820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101524,-1.088429,0.478959,-0.388813,0.155228,0.138917,0.363965,0.247225,-0.124501,-0.458463,-0.492913,...,0.620969,0.608102,0.581827,0.507992,0.360719,0.215012,0.071593,-0.119154,-0.373286,0.258582
101525,-0.160244,-0.220867,-0.232420,-0.235409,-0.231867,-0.171195,-0.070304,-0.010100,0.057378,0.146924,...,-0.936614,-0.656393,-0.225870,0.284487,0.819240,1.257953,1.465637,1.376219,1.073314,0.716656
101526,-0.543972,-1.173676,-1.391533,-1.352929,-1.129094,-0.627628,0.127884,0.892266,1.395359,1.625065,...,-0.961019,-1.262628,-1.478092,-1.373828,-0.943276,-0.404369,0.091978,0.599739,1.099323,1.099323
101527,0.014380,-0.173665,-0.347415,-0.483362,-0.543021,-0.509849,-0.391189,-0.213885,-0.014706,0.177145,...,0.276303,0.171056,0.000233,-0.545690,-0.969520,-0.998621,-0.856398,-0.722979,-0.525605,-0.525605


In [14]:
augmentation_train_y

0         1.0
1         0.0
2         2.0
3         0.0
4         3.0
         ... 
101524    0.0
101525    2.0
101526    0.0
101527    3.0
101528    0.0
Name: 1000, Length: 101529, dtype: float64

In [15]:
from sklearn.preprocessing import StandardScaler

# create a StandardScaler object
scaler = StandardScaler()

# fit the scaler to your dataframe and transform it
augmentation_train_x_std = scaler.fit_transform(augmentation_train_x)
augmentation_train_x = pd.DataFrame(augmentation_train_x_std)

# one-hot encoding
augmentation_Y = pd.get_dummies(augmentation_train_y)
augmentation_Y = augmentation_Y.replace({True: 1, False: 0}).astype(float)

In [16]:
import lightgbm as lgb
from lightgbm import LGBMClassifier
clf_multiclass = LGBMClassifier()
clf_multiclass.fit(augmentation_train_x, augmentation_train_y)
val_pred = clf_multiclass.predict(test_x)

In [17]:
print(accuracy_score(test_set_y,val_pred), np.sum(val_pred))

0.6088934850051706 668.0


In [18]:
from sklearn.metrics import confusion_matrix
NN_confusion_matrix = confusion_matrix(test_set_y, val_pred).T
print(NN_confusion_matrix)

[[2893  429 1090  120]
 [  41   33   21   24]
 [   3    0    0    0]
 [ 142   12    9   18]]


In [21]:
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':4,
    'metric': 'multi_logloss',
    'learning_rate': 0.01,
    'max_depth': 7,
    'num_leaves': 17,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.6,
    'bagging_freq': 17}
d_train = lgb.Dataset(augmentation_train_x, label=augmentation_train_y)
lgb_cv = lgb.cv(params, d_train, num_boost_round=10000, nfold=3, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=100)

nround = lgb_cv['multi_logloss-mean'].index(np.min(lgb_cv['multi_logloss-mean']))
print(nround)

model = lgb.train(params, d_train, num_boost_round=nround)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.213128 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255000
[LightGBM] [Info] Number of data points in the train set: 67686, number of used features: 1000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.196884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255000
[LightGBM] [Info] Number of data points in the train set: 67686, number of used features: 1000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.203495 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255000
[LightGBM] [Info] Number of data points in the train set: 67686, number of used features: 1000
[LightGBM] [Info] Start training from score -0.648846
[LightGBM] [Info] Start training from score -2.080950
[Lig

[2560]	cv_agg's multi_logloss: 0.775367 + 0.00132283
[2580]	cv_agg's multi_logloss: 0.773989 + 0.00131812
[2600]	cv_agg's multi_logloss: 0.772638 + 0.00129002
[2620]	cv_agg's multi_logloss: 0.771301 + 0.00128492
[2640]	cv_agg's multi_logloss: 0.769942 + 0.00130525
[2660]	cv_agg's multi_logloss: 0.768606 + 0.00129665
[2680]	cv_agg's multi_logloss: 0.767266 + 0.00124617
[2700]	cv_agg's multi_logloss: 0.765963 + 0.00127058
[2720]	cv_agg's multi_logloss: 0.764639 + 0.00127372
[2740]	cv_agg's multi_logloss: 0.763307 + 0.00128347
[2760]	cv_agg's multi_logloss: 0.76204 + 0.00132188
[2780]	cv_agg's multi_logloss: 0.760792 + 0.00135864
[2800]	cv_agg's multi_logloss: 0.75956 + 0.00138469
[2820]	cv_agg's multi_logloss: 0.758286 + 0.00142205
[2840]	cv_agg's multi_logloss: 0.757059 + 0.00138414
[2860]	cv_agg's multi_logloss: 0.755717 + 0.00145048
[2880]	cv_agg's multi_logloss: 0.754434 + 0.00148879
[2900]	cv_agg's multi_logloss: 0.753206 + 0.0015442
[2920]	cv_agg's multi_logloss: 0.752012 + 0.00159

[5680]	cv_agg's multi_logloss: 0.633647 + 0.00211947
[5700]	cv_agg's multi_logloss: 0.63307 + 0.00206396
[5720]	cv_agg's multi_logloss: 0.63248 + 0.0020671
[5740]	cv_agg's multi_logloss: 0.631903 + 0.00209612
[5760]	cv_agg's multi_logloss: 0.631331 + 0.00210364
[5780]	cv_agg's multi_logloss: 0.630748 + 0.00209853
[5800]	cv_agg's multi_logloss: 0.630174 + 0.00209629
[5820]	cv_agg's multi_logloss: 0.629577 + 0.00211292
[5840]	cv_agg's multi_logloss: 0.629017 + 0.00212045
[5860]	cv_agg's multi_logloss: 0.628482 + 0.00209819
[5880]	cv_agg's multi_logloss: 0.627939 + 0.00209754
[5900]	cv_agg's multi_logloss: 0.627423 + 0.0021263
[5920]	cv_agg's multi_logloss: 0.626883 + 0.0021174
[5940]	cv_agg's multi_logloss: 0.626317 + 0.00213248
[5960]	cv_agg's multi_logloss: 0.62581 + 0.00215566
[5980]	cv_agg's multi_logloss: 0.625268 + 0.00216628
[6000]	cv_agg's multi_logloss: 0.624778 + 0.00217055
[6020]	cv_agg's multi_logloss: 0.624231 + 0.00213238
[6040]	cv_agg's multi_logloss: 0.623693 + 0.00210449

[8800]	cv_agg's multi_logloss: 0.569231 + 0.00221083
[8820]	cv_agg's multi_logloss: 0.568944 + 0.0022201
[8840]	cv_agg's multi_logloss: 0.568641 + 0.00220242
[8860]	cv_agg's multi_logloss: 0.56839 + 0.00216959
[8880]	cv_agg's multi_logloss: 0.568129 + 0.0021736
[8900]	cv_agg's multi_logloss: 0.567887 + 0.0021799
[8920]	cv_agg's multi_logloss: 0.567602 + 0.0021739
[8940]	cv_agg's multi_logloss: 0.567315 + 0.00215148
[8960]	cv_agg's multi_logloss: 0.567028 + 0.00215469
[8980]	cv_agg's multi_logloss: 0.566782 + 0.00215274
[9000]	cv_agg's multi_logloss: 0.56649 + 0.00213346
[9020]	cv_agg's multi_logloss: 0.566209 + 0.00214549
[9040]	cv_agg's multi_logloss: 0.565938 + 0.00214283
[9060]	cv_agg's multi_logloss: 0.565696 + 0.00213233
[9080]	cv_agg's multi_logloss: 0.565405 + 0.00212494
[9100]	cv_agg's multi_logloss: 0.565115 + 0.00214434
[9120]	cv_agg's multi_logloss: 0.564834 + 0.0021982
[9140]	cv_agg's multi_logloss: 0.56456 + 0.00219128
[9160]	cv_agg's multi_logloss: 0.564328 + 0.00213381
[

In [26]:
preds = model.predict(test_x)
print(preds) 
tmp = np.argmax(preds, axis=1)

[[0.60195107 0.06516317 0.02862592 0.30425983]
 [0.54760456 0.0797097  0.0966598  0.27602595]
 [0.80450448 0.02101062 0.0607405  0.1137444 ]
 ...
 [0.70875599 0.03968137 0.06667306 0.18488957]
 [0.5687513  0.11274975 0.01997947 0.29851948]
 [0.55354164 0.12368433 0.09819259 0.22458144]]


In [27]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_set_y,tmp))

0.59358841778697


In [29]:
from sklearn.metrics import confusion_matrix
NN_confusion_matrix = confusion_matrix(test_set_y, tmp).T
print(NN_confusion_matrix)

[[2793  390 1070  109]
 [  54   62   20   38]
 [   9    0    0    0]
 [ 223   22   30   15]]
